In [30]:
import pandas as pd
from datetime import datetime
import os, re, sqlite3

In [31]:
file = r"C:\Users\BXDM\Downloads\002911历史数据.csv"
df = pd.read_csv(file)
filename = os.path.basename(file)
stock_code = re.search(r'\d+', filename).group()

In [32]:
column_mapping = {
    '日期': 'trade_date',
    '开盘': 'open_price',  # Open: 开盘价
    '收盘': 'close_price',  # LatestPrice: 最新股价
    '高': 'high_price',  # High: 最高价
    '低': 'low_price',  # Low: 最低价
    '交易量': 'volume',  # TotalVolume: 总成交量(/股)
    '涨跌幅': 'pct_change',  # PriceChangeRate: 涨幅百分比
}

In [33]:
df = df[column_mapping.keys()].rename(columns=column_mapping)

In [34]:
# 成交量转换为股数

import re

def convert_volume_to_shares(volume_str):
    """
    将成交量字符串转换为股数（整数）：
    - "万" → 万手 → 股（1万手 = 100万股）
    - "M" → 百万股 → 股（1M = 1,000,000股）
    - 纯数字 → 手 → 股（1手 = 100股）
    """
    if pd.isna(volume_str):  # 处理空值
        return 0
    
    volume_str = str(volume_str).strip()  # 转为字符串并去除空格
    
    # 情况1：带"万"（万手）
    if "万" in volume_str:
        num = float(re.sub(r"[^\d.]", "", volume_str))  # 提取数字（如"241万"→241）
        return int(num * 100 * 10000)  # 万手 → 股
    
    # 情况2：带"M"（百万股）
    elif "M" in volume_str.upper():
        num = float(re.sub(r"[^\d.]", "", volume_str))  # 提取数字（如"1.5M"→1.5）
        return int(num * 1000000)  # 百万股 → 股
    
    # 情况3：纯数字（默认单位是手）
    else:
        try:
            num = float(volume_str)
            return int(num * 100)  # 手 → 股
        except ValueError:
            return 0  # 无效数据返回0
        
# 时间格式转换

def normalize_date(date_str):
    try:
        dt = datetime.strptime(str(date_str), "%Y-%m-%d")
        return dt.strftime("%Y-%m-%d")
    except Exception:
        return ""
# 统一涨跌幅,移除 % 符号
def remove_percent(pct_str):
    try:
        return float(str(pct_str).replace('%', ''))
    except Exception:
        return None


In [35]:
# 数据处理
df['volume'] = df['volume'].apply(convert_volume_to_shares)
df['pct_change'] = df['pct_change'].apply(remove_percent)  # 转换涨跌幅
df['trade_date'] = df['trade_date'].apply(normalize_date)  # 转换日期格式
df["update_time"] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')    # 统一更新时间
df["stock_code"] = stock_code  # 添加股票代码列

In [36]:
df

,trade_date,open_price,close_price,high_price,low_price,volume,pct_change,update_time,stock_code
0,2025-07-15,10.55,10.43,10.60,10.36,4010000,-1.23,2025-07-22 22:16:02,002911
1,2025-07-14,10.50,10.56,10.61,10.41,7190000,1.73,2025-07-22 22:16:02,002911
2,2025-07-11,10.28,10.38,10.69,10.28,10770000,1.37,2025-07-22 22:16:02,002911
3,2025-07-10,10.23,10.24,10.30,10.21,2890000,0.10,2025-07-22 22:16:02,002911
4,2025-07-09,10.20,10.23,10.26,10.19,2550000,0.29,2025-07-22 22:16:02,002911
...,...,...,...,...,...,...,...,...,...
607,2023-01-09,10.01,10.59,10.71,9.99,25200000,1.85,2025-07-22 22:16:02,002911
608,2023-01-06,10.23,10.39,10.88,10.08,42410000,1.58,2025-07-22 22:16:02,002911
609,2023-01-05,9.39,10.23,10.23,9.28,18780000,10.01,2025-07-22 22:16:02,002911
610,2023-01-04,9.42,9.30,9.42,9.22,6530000,-1.23,2025-07-22 22:16:02,002911


In [37]:
import sqlite3

conn = sqlite3.connect('stock_data.db')

update_fields = [col for col in df.columns if col not in ['stock_code', 'trade_date']]
set_clause = ', '.join([f"{col}=excluded.{col}" for col in update_fields])
placeholders = ', '.join(['?'] * len(df.columns))

sql = f"""
INSERT INTO stock_daily_data ({', '.join(df.columns)})
VALUES ({placeholders})
ON CONFLICT(stock_code, trade_date) DO UPDATE SET
{set_clause}
"""

conn.executemany(sql, df.values.tolist())
conn.commit()
conn.close()